| Feature               | `Flat`                | `IVF` (Inverted File Index)        | `HNSW` (Graph-based Index)          |
| --------------------- | --------------------- | ---------------------------------- | ----------------------------------- |
| Type of Search     | Exact                 | Approximate (cluster-based)        | Approximate (graph-based traversal) |
| Speed               | Slow (linear scan)    | Fast (search only in top clusters) | Very Fast (graph walk)              |


| Dataset Size              | Recommended Index                 |
| ------------------------- | --------------------------------- |
| UPTO 1L                     | `IndexFlatL2` or `IndexFlatIP`    |
| UPTO 1M                  | `IndexIVFFlat` or `IndexHNSWFlat` |
| > 1M                      | `IndexIVFPQ` or `IndexHNSWFlat`   |


In [1]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents=[
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\vijay\work\code\AI\AgenticAILearning\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [7]:
index=faiss.IndexFlatL2(384)

vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [8]:
vector_store.add_documents(documents=documents)

['632f168a-66e9-4422-9ebd-01f887f49a53',
 'f4afa37a-4b90-436a-87b8-ce0a35035ff1',
 'd1124f5a-1eb6-4892-8c43-5d1222fe7646',
 '03721196-e09b-4b5e-8409-c121a618a83b',
 '260f16ac-4196-4e6a-8002-d6898aad11b4',
 'e046a48a-bc66-4fd6-aef0-8b65c41d48ee',
 '7265c12b-58c9-4d71-b10d-d33d0c4f8dad',
 '5070cd01-fabb-4c02-9464-abe35905b0ca',
 'd6536a11-aa55-41fb-b3ea-3aa55c52be5c',
 'c095787d-b4f4-408a-934e-b93c687ce6ee']

In [9]:
vector_store.similarity_search("LangChain provides abstractions to make working with LLMs easy",k=2)

[Document(id='d1124f5a-1eb6-4892-8c43-5d1222fe7646', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='5070cd01-fabb-4c02-9464-abe35905b0ca', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!')]

In [11]:
result=vector_store.similarity_search("LangChain provides abstractions to make working with LLMs easy", filter={"source": "tweet"})
result

[Document(id='d1124f5a-1eb6-4892-8c43-5d1222fe7646', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='5070cd01-fabb-4c02-9464-abe35905b0ca', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='c095787d-b4f4-408a-934e-b93c687ce6ee', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :('),
 Document(id='632f168a-66e9-4422-9ebd-01f887f49a53', metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.')]

In [12]:
result[0].metadata

{'source': 'tweet'}

### Lest build a retriver now

In [14]:
retriever=vector_store.as_retriever(search_kwargs={"k":3})

In [15]:
retriever.invoke("LangChain provides abstractions to make working with LLMs easy")

[Document(id='d1124f5a-1eb6-4892-8c43-5d1222fe7646', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='5070cd01-fabb-4c02-9464-abe35905b0ca', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='c095787d-b4f4-408a-934e-b93c687ce6ee', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :(')]

#### now store the faiss indext to local disk

In [16]:
vector_store.save_local("faiss_index")

In [18]:
local_vector_store=FAISS.load_local("faiss_index",embeddings=embeddings,allow_dangerous_deserialization=True)

In [19]:
local_vector_store.similarity_search("Langchain")

[Document(id='d1124f5a-1eb6-4892-8c43-5d1222fe7646', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='5070cd01-fabb-4c02-9464-abe35905b0ca', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='260f16ac-4196-4e6a-8002-d6898aad11b4', metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again."),
 Document(id='7265c12b-58c9-4d71-b10d-d33d0c4f8dad', metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.')]